This Notebook is a kaggle tutorial for Japanese kaggle beginners writen in Japanese.

# 6. submitのその前に！ 「Cross Validation」の大切さを知ろう

3〜5つ目のNotebookでは、特徴量エンジニアリング・機械学習アルゴリズム・ハイパーパラメータの面で、スコアを上げていく方法を学びました。

この[Notebook](https://www.kaggle.com/sishihara/upura-kaggle-tutorial-06-validation)では、機械学習モデルの性能を見積もる「validation」について解説します。

In [ ]:
# 特徴量の準備

import numpy as np
import pandas as pd

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

特徴量の準備が完了しました。

# ホールドアウト検証

実は既に、LightGBMを利用する段階で「ホールドアウト検証」と呼ばれる一種のvalidationを実施していました。学習用データセットを分割した上でLightGBMを学習させていたことを思い出してください。

この検証用データセットは、自分で学習用データセットから切り出しているため、目的変数を含めて全容を把握できています。全体像の見えていないpublic LBのスコアに比べて、信頼性の高いスコアを得られる可能性を秘めています。

検証用データセットに対する性能、すなわちvalidationスコアは、提出することなく手元で確認可能です。自分の気の済むだけ試行錯誤を回し、良いスコアを得た場合に実際にKaggleに提出するような運用が可能です。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

In [ ]:
# カテゴリ変数の指定
categorical_features = ['Embarked', 'Pclass', 'Sex']

In [ ]:
params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

In [ ]:
sub = gender_submission

sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm_holdout.csv", index=False)

sub.head()

この予測値は、私の環境で0.77033というスコアが出ています（前回のKernel参照）。

# 交差検証（Cross Validation）

さて、ここで「Cross Validation」を実行すると、ホールドアウト検証の例よりも、更に汎用的に性能を確認できます。交差検証（Cross Validation）とは、複数回にわたって異なる方法でデータセットを分割し、それぞれでホールドアウト検証を実行する方法です。そのスコアの平均を確認することで、1回のホールドアウト検証で生じうる偏りに対する懸念を弱めることができます。

`train_test_split` を複数回書いて実装することもできそうですが、より便利なパッケージが用意されています。

In [ ]:
# 先にtrain_test_splitしてしまっているので、改めて特徴量の準備

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

X_train.head()

In [ ]:
from sklearn.model_selection import KFold


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

Cross Validationを実施した際は、全体の平均をvalidationスコアと見なすことが多いです。このスコアのことを「CVスコア」、省略して単に「CV」とも呼びます。

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
len(y_preds)

In [ ]:
y_preds[0][:10]

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

In [ ]:
sub['Survived'] = y_sub
sub.to_csv("submission_lightgbm_kfold.csv", index=False)

sub.head()

この予測値を提出すると、私の環境で0.76555というスコアが出ました。ホールドアウト検証の時よりも悪いスコアになってしまいました。その原因の一つは、データセットの分割方法だと推察されます。最後に、この部分を掘り下げて解説していきます。

# データセットの分割方法

データセットの分割に当たっては、データセットや課題設定の特徴を意識するのが大切です。

ここまで使っていたKFoldは、特にデータセットや課題設定の特徴を考慮することなくデータセットを分割します。例えば、学習用・検証用データセット内の`y==1`の予測値の割合を見てみると次のようになりました。fold: 2, 4などで顕著に割合が異なっていると分かります。

In [ ]:
from sklearn.model_selection import KFold


cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

繰り返しになりますが、Kaggleの目的は未知のデータセットであるLBに対する性能を高めることです。未知のデータセットにおける`y==1`の割合は誰にも正確には分からないですが、既存のデータセットである学習用データセットと同様の割合だと近似するのが一般的です。つまり、データセットは`y==1`の割合を保つように分割するのが理想的です。

`y==1`の割合が均等でない場合、`y==1`を重要視したり逆に軽視したりと、機械学習アルゴリズムの学習がうまくいかない傾向にあります。このような状況では適切に特徴を学習できず、未知のデータセットに対する性能が劣化してしまう可能性があります。KFoldを用いた場合にスコアが悪化した原因もここにあると考えられます。

In [ ]:
from sklearn.model_selection import StratifiedKFold


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

この分割を用いて学習・予測を実行しましょう。

In [ ]:
from sklearn.model_selection import StratifiedKFold


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_skfold.csv', index=False)
print(oof_train[:10])

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

In [ ]:
sub['Survived'] = y_sub
sub.to_csv("submission_lightgbm_skfold.csv", index=False)

sub.head()

この予測値を提出すると、私の環境で0.77511というスコアが出ました。`KFold`の時よりも、ホールドアウト検証の時よりも良いスコアが出ています。